# Functional Class and Curvature Analysis
This notebook illustrates the complete analysis example using Here Map Content (HMC) attributes.

### Dependencies
* Catalogs: [rib-2](https://platform.here.com/data/hrn:here:data::olp-here:rib-2)
* Layers: [Topology Geometry](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/topology-geometry), [Road Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/road-attributes), [ADAS Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/adas-attributes)

### Workflow
* Start with a bounding box that contains portions of two tiles
* Extract geometry for the two tiles
* Keep only the segments that are (at least partially) in original bounding box
* Extract Functional Class (FC) attributes for those segments
* Select segments with FC = 3, 4, and 5
* Extract curvature for the survived segments
* Analyze average curvature
* Drop segments below the average curvature

## Set API key (here.access.key.secret)

In [1]:
import os
os.environ["LS_API_KEY"] = "your access key secret for HERE platform"

## Get partition ID's from a given bounding box

In [2]:
from here.geotiles.heretile import in_bounding_box
from shapely.geometry import Point

zoom_level = 12
west = 13.41
east = 13.49
south = 52.50
north = 52.54
center = Point(west, south)
tileIds = list(in_bounding_box(west=west, south=south, east=east, north=north, level=zoom_level))

# partition ID's: string representation of tile IDs - this will be used for extracting attributes later
partition_ids = [str(tile) for tile in tileIds]
print("partition ids:", partition_ids)

partition ids: ['23618402', '23618403']


## Visualize the bounding box and tile boundaries
- bounding box: Red
- tile boundaries: Gray

In [3]:
from here.inspector import inspect
from here.inspector.styles import Color
from shapely.geometry import Polygon

# construct a bounding box geojson
bb_polygon = Polygon([(west, south), (west, north), (east, north), (east, south), (west, south)])

# create inspect object with tile boundaries and the bounding box
inspect(layers={"bounding_box": [bb_polygon]},
        tiles=tileIds,
        center=center,
        zoom=11,
        layers_style={"bounding_box": Color.RED},
        tiles_style=Color.GRAY)

Map(api_key='307FLRnKO9vTTcbKgi1Uga3F_MHPVRvw3bbvQVQNfxfXXqk0KIoQ-XE3wW11AlIn7UKzRqR3DdVr3kYYib_pjQ', basemap=…

## Extract topology geometry and clip by bounding box

In [4]:
from here.platform import Platform
from here.content.hmc2.hmc import HMC
from here.geopandas_adapter import GeoPandasAdapter
import pandas as pd
import geopandas as gpd

# create HMC object
hmc = HMC(Platform(), adapter=GeoPandasAdapter())

In [5]:
# get segments dataframe from given tiles
segment_df = hmc.topology_geometry.get(tileIds, "segment")
segment_df

start_node_ref.partition  \
partition_id segment_id                                           
23618402     here:cm:segment:100005273                 23618402   
             here:cm:segment:100005274                 23618402   
             here:cm:segment:100005285                 23618402   
             here:cm:segment:100005286                 23618402   
             here:cm:segment:100005287                 23618402   
...                                                         ...   
23618403     here:cm:segment:99973024                  23618403   
             here:cm:segment:99973086                  23618403   
             here:cm:segment:99983233                  23618403   
             here:cm:segment:99990525                  23618403   
             here:cm:segment:99994957                  23618403   

                                       start_node_ref.identifier  \
partition_id segment_id                                            
23618402     here:cm:segment:100005273    here:cm:node:850753878   
             here:cm:segment:100005274    here:cm:node:850753876   
             here:cm:segment:100005285    here:cm:node:850806348   
             here:cm:segment:100005286     here:cm:node:61051173   
             here:cm:segment:100005287    here:cm:node:850806349   
...                                                          ...   
23618403     here:cm:segment:99973024      here:cm:node:61081169   
             here:cm:segment:99973086      here:cm:node:61042975   
             here:cm:segment:99983233      here:cm:node:61080290   
             here:cm:segment:99990525     here:cm:node:952743094   
             here:cm:segment:99994957      here:cm:node:61061133   

                                       end_node_ref.partition  \
partition_id segment_id                                         
23618402     here:cm:segment:100005273               23618402   
             here:cm:segment:100005274               23618402   
             here:cm:segment:100005285               23618402   
             here:cm:segment:100005286               23618402   
             here:cm:segment:100005287               23618402   
...                                                       ...   
23618403     here:cm:segment:99973024                23618403   
             here:cm:segment:99973086                23618403   
             here:cm:segment:99983233                23618403   
             here:cm:segment:99990525                23618403   
             here:cm:segment:99994957                23618403   

                                       end_node_ref.identifier  \
partition_id segment_id                                          
23618402     here:cm:segment:100005273   here:cm:node:61059000   
             here:cm:segment:100005274  here:cm:node:850753878   
             here:cm:segment:100005285  here:cm:node:850806350   
             here:cm:segment:100005286  here:cm:node:850806349   
             here:cm:segment:100005287   here:cm:node:61051152   
...                                                        ...   
23618403     here:cm:segment:99973024    here:cm:node:61081168   
             here:cm:segment:99973086    here:cm:node:61042976   
             here:cm:segment:99983233    here:cm:node:61080287   
             here:cm:segment:99990525   here:cm:node:952743093   
             here:cm:segment:99994957    here:cm:node:61061134   

                                                                                 geometry  \
partition_id segment_id                                                                     
23618402     here:cm:segment:100005273  LINESTRING (13.44399 52.51531, 13.44405 52.515...   
             here:cm:segment:100005274  LINESTRING (13.44347 52.51514, 13.44399 52.51531)   
             here:cm:segment:100005285  LINESTRING (13.40016 52.49682, 13.39965 52.496...   
             here:cm:segment:100005286  LINESTRING (13.40222 52.49641, 13.40076 52.49645)   
             here:c

### Clip segments by the bounding box
Since the geometry objects for the segment dataframe is LineString, the intersection method will do the clipping. See the documentation of shapely: https://shapely.readthedocs.io/en/stable/manual.html

In [6]:
clipped_segments = segment_df.intersection(bb_polygon)
segment_in_bb = segment_df[clipped_segments.apply(lambda x: x.length != 0)]
segment_in_bb

start_node_ref.partition  \
partition_id segment_id                                           
23618402     here:cm:segment:100005273                 23618402   
             here:cm:segment:100005274                 23618402   
             here:cm:segment:100036943                 23618402   
             here:cm:segment:100036944                 23618402   
             here:cm:segment:100095630                 23618402   
...                                                         ...   
23618403     here:cm:segment:99912978                  23618403   
             here:cm:segment:99966244                  23618403   
             here:cm:segment:99966357                  23618403   
             here:cm:segment:99973008                  23618403   
             here:cm:segment:99994957                  23618403   

                                       start_node_ref.identifier  \
partition_id segment_id                                            
23618402     here:cm:segment:100005273    here:cm:node:850753878   
             here:cm:segment:100005274    here:cm:node:850753876   
             here:cm:segment:100036943    here:cm:node:828712981   
             here:cm:segment:100036944    here:cm:node:850831168   
             here:cm:segment:100095630    here:cm:node:850843823   
...                                                          ...   
23618403     here:cm:segment:99912978     here:cm:node:858502534   
             here:cm:segment:99966244      here:cm:node:61062298   
             here:cm:segment:99966357      here:cm:node:61061654   
             here:cm:segment:99973008      here:cm:node:61062264   
             here:cm:segment:99994957      here:cm:node:61061133   

                                       end_node_ref.partition  \
partition_id segment_id                                         
23618402     here:cm:segment:100005273               23618402   
             here:cm:segment:100005274               23618402   
             here:cm:segment:100036943               23618402   
             here:cm:segment:100036944               23618402   
             here:cm:segment:100095630               23618402   
...                                                       ...   
23618403     here:cm:segment:99912978                23618403   
             here:cm:segment:99966244                23618403   
             here:cm:segment:99966357                23618403   
             here:cm:segment:99973008                23618403   
             here:cm:segment:99994957                23618403   

                                       end_node_ref.identifier  \
partition_id segment_id                                          
23618402     here:cm:segment:100005273   here:cm:node:61059000   
             here:cm:segment:100005274  here:cm:node:850753878   
             here:cm:segment:100036943  here:cm:node:850831168   
             here:cm:segment:100036944  here:cm:node:810808453   
             here:cm:segment:100095630   here:cm:node:61059309   
...                                                        ...   
23618403     here:cm:segment:99912978   here:cm:node:858502517   
             here:cm:segment:99966244    here:cm:node:61062299   
             here:cm:segment:99966357    here:cm:node:61061647   
             here:cm:segment:99973008    here:cm:node:61062265   
             here:cm:segment:99994957    here:cm:node:61061134   

                                                                                 geometry  \
partition_id segment_id                                                                     
23618402     here:cm:segment:100005273  LINESTRING (13.44399 52.51531, 13.44405 52.515...   
             here:cm:segment:100005274  LINESTRING (13.44347 52.51514, 13.44399 52.51531)   
             here:cm:segment:100036943  LINESTRING (13.42226 52.52790, 13.42192 52.528...   
             here:cm:segment:100036944  LINESTRING (13.42179 52.52816, 13.42143 52.528...   
             here:c

In [7]:
# show the clipped segments and original tile boundaries
inspect(features=segment_in_bb["geometry"], tiles=tileIds, tiles_style=Color.GRAY)

Map(api_key='307FLRnKO9vTTcbKgi1Uga3F_MHPVRvw3bbvQVQNfxfXXqk0KIoQ-XE3wW11AlIn7UKzRqR3DdVr3kYYib_pjQ', basemap=…

## Get functional class attribute with segment reference

In [8]:
# get functional_class attributes
fc_df_seg = hmc.road_attributes.get_referencing(tileIds, "functional_class", "segment")
fc_df_seg

segments  \
ref_partition ref_identifier                                                                 
23618402      here:cm:segment:209900713  [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:209900713  [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:209900713  [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:209900713  [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:209900713  [{'ref': {'partition': '23618402', 'identifier...   
...                                                                                    ...   
23618403      here:cm:segment:887576350  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887579808  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887579808  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887678602  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887678602  [{'ref': {'partition': '23618403', 'identifier...   

                                         forward  backward  \
ref_partition ref_identifier                                 
23618402      here:cm:segment:209900713     True      True   
              here:cm:segment:209900713     True      True   
              here:cm:segment:209900713     True      True   
              here:cm:segment:209900713     True      True   
              here:cm:segment:209900713     True      True   
...                                          ...       ...   
23618403      here:cm:segment:887576350     True      True   
              here:cm:segment:887579808     True      True   
              here:cm:segment:887579808     True      True   
              here:cm:segment:887678602     True      True   
              here:cm:segment:887678602     True      True   

                                        attribute.functional_class  
ref_partition ref_identifier                                        
23618402      here:cm:segment:209900713         FUNCTIONAL_CLASS_1  
              here:cm:segment:209900713         FUNCTIONAL_CLASS_1  
              here:cm:segment:209900713         FUNCTIONAL_CLASS_1  
              here:cm:segment:209900713         FUNCTIONAL_CLASS_1  
              here:cm:segment:209900713         FUNCTIONAL_CLASS_1  
...                                                            ...  
23618403      here:cm:segment:887576350         FUNCTIONAL_CLASS_5  
              here:cm:segment:887579808         FUNCTIONAL_CLASS_5  
              here:cm:segment:887579808         FUNCTIONAL_CLASS_5  
              here:cm:segment:887678602         FUNCTIONAL_CLASS_5  
              here:cm:segment:887678602         FUNCTIONAL_CLASS_5  

[35478 rows x 4 columns]

## Select FC = 3, 4, or 5

In [9]:
# select FC = 3, 4, 5
fc345_df = fc_df_seg[
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_3") |
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_4") |
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_5")
]
fc345_df

segments  \
ref_partition ref_identifier                                                                 
23618402      here:cm:segment:76338653   [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:76338653   [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:76338653   [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:76365186   [{'ref': {'partition': '23618402', 'identifier...   
              here:cm:segment:76365192   [{'ref': {'partition': '23618402', 'identifier...   
...                                                                                    ...   
23618403      here:cm:segment:887576350  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887579808  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887579808  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887678602  [{'ref': {'partition': '23618403', 'identifier...   
              here:cm:segment:887678602  [{'ref': {'partition': '23618403', 'identifier...   

                                         forward  backward  \
ref_partition ref_identifier                                 
23618402      here:cm:segment:76338653      True      True   
              here:cm:segment:76338653      True      True   
              here:cm:segment:76338653      True      True   
              here:cm:segment:76365186      True      True   
              here:cm:segment:76365192      True      True   
...                                          ...       ...   
23618403      here:cm:segment:887576350     True      True   
              here:cm:segment:887579808     True      True   
              here:cm:segment:887579808     True      True   
              here:cm:segment:887678602     True      True   
              here:cm:segment:887678602     True      True   

                                        attribute.functional_class  
ref_partition ref_identifier                                        
23618402      here:cm:segment:76338653          FUNCTIONAL_CLASS_3  
              here:cm:segment:76338653          FUNCTIONAL_CLASS_3  
              here:cm:segment:76338653          FUNCTIONAL_CLASS_3  
              here:cm:segment:76365186          FUNCTIONAL_CLASS_3  
              here:cm:segment:76365192          FUNCTIONAL_CLASS_3  
...                                                            ...  
23618403      here:cm:segment:887576350         FUNCTIONAL_CLASS_5  
              here:cm:segment:887579808         FUNCTIONAL_CLASS_5  
              here:cm:segment:887579808         FUNCTIONAL_CLASS_5  
              here:cm:segment:887678602         FUNCTIONAL_CLASS_5  
              here:cm:segment:887678602         FUNCTIONAL_CLASS_5  

[33805 rows x 4 columns]

In [10]:
# change indexes into actual columns
fc345_df = fc345_df.reset_index()
# rename columns for easier merging later on
fc345_df.rename(columns={'ref_partition': 'partition_id'}, inplace=True)
fc345_df.rename(columns={'ref_identifier': 'segment_id'}, inplace=True)
fc345_df

,partition_id,segment_id,segments,forward,backward,attribute.functional_class
0,23618402,here:cm:segment:76338653,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_3
1,23618402,here:cm:segment:76338653,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_3
2,23618402,here:cm:segment:76338653,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_3
3,23618402,here:cm:segment:76365186,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_3
4,23618402,here:cm:segment:76365192,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_3
...,...,...,...,...,...,...
33800,23618403,here:cm:segment:887576350,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5
33801,23618403,here:cm:segment:887579808,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5
33802,23618403,here:cm:segment:887579808,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5
33803,23618403,here:cm:segment:887678602,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5


In [11]:
# join two dataframe by partitionId, segmentId
segment_fc345_df = segment_in_bb.merge(fc345_df,
                                       how="inner",
                                       on=["partition_id", "segment_id"])
segment_fc345_df

,partition_id,segment_id,start_node_ref.partition,start_node_ref.identifier,end_node_ref.partition,end_node_ref.identifier,geometry,z_level,length,segments,forward,backward,attribute.functional_class
0,23618402,here:cm:segment:100005273,23618402,here:cm:node:850753878,23618402,here:cm:node:61059000,"LINESTRING (13.44399 52.51531, 13.44405 52.515...","[0, 0, 0]",53.97453,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5
1,23618402,here:cm:segment:100005274,23618402,here:cm:node:850753876,23618402,here:cm:node:850753878,"LINESTRING (13.44347 52.51514, 13.44399 52.51531)","[0, 0]",40.05030,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5
2,23618402,here:cm:segment:100036943,23618402,here:cm:node:828712981,23618402,here:cm:node:850831168,"LINESTRING (13.42226 52.52790, 13.42192 52.528...","[0, 0, 0]",43.06557,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5
3,23618402,here:cm:segment:100036944,23618402,here:cm:node:850831168,23618402,here:cm:node:810808453,"LINESTRING (13.42179 52.52816, 13.42143 52.528...","[0, 0, 0, 0]",72.90009,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5
4,23618402,here:cm:segment:100105178,23618402,here:cm:node:61057576,23618402,here:cm:node:850815518,"LINESTRING (13.41467 52.50976, 13.41386 52.51013)","[0, 0]",68.69943,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8603,23618403,here:cm:segment:99912978,23618403,here:cm:node:858502534,23618403,here:cm:node:858502517,"LINESTRING (13.46656 52.53448, 13.46684 52.53476)","[0, 0]",36.49382,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5
8604,23618403,here:cm:segment:99966244,23618403,here:cm:node:61062298,23618403,here:cm:node:61062299,"LINESTRING (13.47027 52.53993, 13.47023 52.54000)","[0, 0]",8.24868,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_3
8605,23618403,here:cm:segment:99966357,23618403,here:cm:node:61061654,23618403,here:cm:node:61061647,"LINESTRING (13.48814 52.50769, 13.48664 52.50791)","[0, 0]",104.74824,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5
8606,23618403,here:cm:segment:99973008,23618403,here:cm:node:61062264,23618403,here:cm:node:61062265,"LINESTRING (13.47133 52.53259, 13.47081 52.53353)","[0, 0]",110.39244,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5


## Show the segments with FC = 3, 4, or 5

In [12]:
inspect(segment_fc345_df["geometry"])

Map(api_key='307FLRnKO9vTTcbKgi1Uga3F_MHPVRvw3bbvQVQNfxfXXqk0KIoQ-XE3wW11AlIn7UKzRqR3DdVr3kYYib_pjQ', basemap=…

## Show different colors for each FC

In [13]:
fcs = {}
for fc, gdf in segment_fc345_df.groupby("attribute.functional_class"):
    fcs[fc] = gdf["geometry"]
inspect(layers=fcs)

Map(api_key='307FLRnKO9vTTcbKgi1Uga3F_MHPVRvw3bbvQVQNfxfXXqk0KIoQ-XE3wW11AlIn7UKzRqR3DdVr3kYYib_pjQ', basemap=…

## Extract curvatures

In [14]:
# get segment curvatures (curvatures at the nodes are not used in this notebook example)
segment_curvature_df = hmc.adas_attributes.get(partition_ids, "curvature_heading")
segment_curvature_df

,segment_anchors,curvature,heading
partition_id,,,
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-581331,185369
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-512203,51018
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-487694,68208
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-449294,81368
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-449064,286569
...,...,...,...
23618403,"[SegmentAnchor(partition_id='23618403', orient...",356760,258285
23618403,"[SegmentAnchor(partition_id='23618403', orient...",370129,22519
23618403,"[SegmentAnchor(partition_id='23618403', orient...",547620,344059


## Select segments with high curvatures

In [15]:
# create a column called abs_curvature to calculate average curvature
segment_curvature_df["abs_curvature"] = segment_curvature_df.apply(lambda x: abs(x["curvature"]), axis=1)
average_curvature = segment_curvature_df["abs_curvature"].mean()

# select segments with curvature higher than average
highcurv_segments = segment_curvature_df[segment_curvature_df["abs_curvature"] > average_curvature]

# extract segment_id from segment anchors
highcurv_segments = highcurv_segments.copy()
highcurv_segments["segment_id"] = highcurv_segments.apply(
    lambda x: x["segment_anchors"][0].oriented_segment_ref[0][0].segment_id,
    axis=1
)

highcurv_segments

,segment_anchors,curvature,heading,abs_curvature,segment_id
partition_id,,,,,
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-581331,185369,581331,here:cm:segment:146671023
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-512203,51018,512203,here:cm:segment:104572126
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-487694,68208,487694,here:cm:segment:847729117
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-449294,81368,449294,here:cm:segment:90627216
23618402,"[SegmentAnchor(partition_id='23618402', orient...",-449064,286569,449064,here:cm:segment:206150699
...,...,...,...,...,...
23618403,"[SegmentAnchor(partition_id='23618403', orient...",356760,258285,356760,here:cm:segment:863617329
23618403,"[SegmentAnchor(partition_id='23618403', orient...",370129,22519,370129,here:cm:segment:796959311
23618403,"[SegmentAnchor(partition_id='23618403', orient...",547620,344059,547620,here:cm:segment:800999575


## Join two dataframes
Inner join between the dataframe with FC=3,4,5 and the dataframe with high curvatures

In [16]:
segment_fc345_highcurv_df = segment_fc345_df.merge(highcurv_segments,
                                                   how="inner",
                                                   on=["partition_id", "segment_id"])
segment_fc345_highcurv_df

,partition_id,segment_id,start_node_ref.partition,start_node_ref.identifier,end_node_ref.partition,end_node_ref.identifier,geometry,z_level,length,segments,forward,backward,attribute.functional_class,segment_anchors,curvature,heading,abs_curvature
0,23618402,here:cm:segment:100036944,23618402,here:cm:node:850831168,23618402,here:cm:node:810808453,"LINESTRING (13.42179 52.52816, 13.42143 52.528...","[0, 0, 0, 0]",72.90009,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618402', orient...",-34734,341715,34734
1,23618402,here:cm:segment:100036944,23618402,here:cm:node:850831168,23618402,here:cm:node:810808453,"LINESTRING (13.42179 52.52816, 13.42143 52.528...","[0, 0, 0, 0]",72.90009,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618402', orient...",-29617,2157,29617
2,23618402,here:cm:segment:100036944,23618402,here:cm:node:850831168,23618402,here:cm:node:810808453,"LINESTRING (13.42179 52.52816, 13.42143 52.528...","[0, 0, 0, 0]",72.90009,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618402', orient...",-25804,324582,25804
3,23618402,here:cm:segment:100115715,23618402,here:cm:node:850815516,23618402,here:cm:node:850815517,"LINESTRING (13.41399 52.50978, 13.41368 52.50988)","[0, 0]",23.80802,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618402', orient...",-73678,316653,73678
4,23618402,here:cm:segment:100134058,23618402,here:cm:node:850794075,23618402,here:cm:node:850794077,"LINESTRING (13.41857 52.52754, 13.41782 52.52777)","[0, 0]",56.97302,"[{'ref': {'partition': '23618402', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618402', orient...",47464,275343,47464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,23618403,here:cm:segment:99902313,23618403,here:cm:node:858502528,23618403,here:cm:node:858502521,"LINESTRING (13.46219 52.53528, 13.46170 52.535...","[0, 0, 0, 0, 0]",206.71945,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618403', orient...",-15371,314806,15371
5276,23618403,here:cm:segment:99902316,23618403,here:cm:node:963124182,23618403,here:cm:node:858502533,"LINESTRING (13.46217 52.53298, 13.46258 52.533...","[0, 0, 0, 0, 0, 0, 0]",127.83702,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618403', orient...",14484,1363,14484
5277,23618403,here:cm:segment:99902317,23618403,here:cm:node:61062096,23618403,here:cm:node:858502751,"LINESTRING (13.45815 52.53575, 13.45842 52.53586)","[0, 0]",22.03371,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618403', orient...",-22341,75041,22341
5278,23618403,here:cm:segment:99902318,23618403,here:cm:node:858502751,23618403,here:cm:node:858502521,"LINESTRING (13.45842 52.53586, 13.45905 52.535...","[0, 0, 0, 0]",89.86478,"[{'ref': {'partition': '23618403', 'identifier...",True,True,FUNCTIONAL_CLASS_5,"[SegmentAnchor(partition_id='23618403', orient...",18974,76863,18974


In [17]:
# drop abs_curvature which was used to calculate average curvature
merged_df = segment_fc345_highcurv_df.drop("abs_curvature", axis=1)
merged_df.columns

Index(['partition_id', 'segment_id', 'start_node_ref.partition',
       'start_node_ref.identifier', 'end_node_ref.partition',
       'end_node_ref.identifier', 'geometry', 'z_level', 'length', 'segments',
       'forward', 'backward', 'attribute.functional_class', 'segment_anchors',
       'curvature', 'heading'],
      dtype='object')

## Show the survived segments

In [18]:
inspect(merged_df["geometry"])

Map(api_key='307FLRnKO9vTTcbKgi1Uga3F_MHPVRvw3bbvQVQNfxfXXqk0KIoQ-XE3wW11AlIn7UKzRqR3DdVr3kYYib_pjQ', basemap=…

<span style="float:left; margin-top:3px;"><img src="https://www.here.com/themes/custom/here_base_theme_v2/logo.svg" alt="HERE Logo" height="60" width="60"></span><span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2021 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential information which may not be disclosed to others without the prior written consent of HERE.</sub></span>